In [1]:
from bqplot import (
    Axis, Scatter, Lines, Figure, LinearScale
)
from ipywidgets import HBox
from collections import namedtuple
from numpy import add
sc_x = LinearScale(min=-5,max=5)
sc_y = sc_x

Vector = namedtuple('Vector',['vec','marks','addListeners'])
def vector(v,color='green'): 
    """
    An interactive vector.
    @return the marks making it up, and the method to update self
    """
    x_a,y_a = v
    lin = Lines(x=[0,x_a], y=[0,y_a], scales={'x': sc_x, 'y': sc_y}, colors=[color])
    head = Scatter(x=[x_a], y=[y_a], scales={'x': sc_x, 'y': sc_y}, colors=[color],enable_move=True)
    update_funcs=[] #list of functions taking x,y as input to be called when this vector is changed
    def vec():
        return (head.x[0],head.y[0])
    p = Vector(vec=vec,marks=[lin,head],addListeners=lambda x: update_funcs.append(x))
    def update_self():
        lin.x=[0,head.x[0]]
        lin.y=[0,head.y[0]]
    head.update_on_move=True
    def update(change=None):
        update_self()
        for f in update_funcs:
            f(head.x[0], head.y[0])
    head.observe(update,names=['x','y'])
    return p

# TODO: add arrow / icon indicating the direction of the surface 
def par(off_v,i_hat,j_hat,color):
    """
    A parallelogram.
    """
    x_off,y_off=off_v
    i_x,i_y=i_hat
    j_x,j_y=j_hat
    lin = Lines(  x=[], y=[]
                , scales={'x': sc_x, 'y': sc_y}
                , close_path=True, fill='inside'
                , fill_opacities=[0.5]
                , colors=[color])
    def update(off_vn=None,i_hatn=None,j_hatn=None):#parameter defaults are not dynamic!
        nonlocal off_v,i_hat,j_hat
        off_v,i_hat,j_hat=off_v if off_vn is None else off_vn,i_hat if i_hatn is None else i_hatn,j_hat if j_hatn is None else j_hatn
        (x_off,y_off),(i_x,i_y),(j_x,j_y)=off_v,i_hat,j_hat
        lin.y=[y+y_off for y in [0,i_y,i_y+j_y,j_y]]
        lin.x=[x+x_off for x in [0,i_x,i_x+j_x,j_x]]
    update()
    return (lin,update)

# Determinant Sum Demonstration

In [3]:
i_1 = vector((1,0),'green') #a
j_1= vector((0,1),'orange') #b
par1,u_par1 = par((0,0),i_1.vec(),j_1.vec(),'green') #|a b|

i_2 = vector((0,1),'blue') #c
j_2= vector((1,0),'red') #d
par2,u_par2 = par(add(i_1.vec(),j_1.vec()),i_2.vec(),j_2.vec(),'blue') #|c dQ|

par3,u_par3 = par(j_1.vec(),i_1.vec(),j_2.vec(),'red') #|a d|
par4,u_par4 = par(i_1.vec(),i_2.vec(),j_1.vec(),'orange')#|c b|

#par for actual sum
par5,u_par5 = par((0,0),add(i_1.vec(),i_2.vec()),add(j_1.vec(),j_2.vec()),'white')


i_1.addListeners(lambda x, y: ( u_par1(i_hatn=(x,y)) #this screams it needs improvement. incorporate bindings directly into the 'gram definition somehow?
                               ,u_par2(off_vn=add(j_1.vec(),(x,y)))
                               ,u_par3(i_hatn=(x,y))
                               ,u_par4(off_vn=(x,y))
                               ,u_par5(i_hatn=add((x,y),i_2.vec()))
                              ))
j_1.addListeners(lambda x, y: ( u_par1(j_hatn=(x,y)), u_par2(off_vn=add((x,y),i_1.vec()))
                               ,u_par2(off_vn=add(i_1.vec(),(x,y)))
                               ,u_par3(off_vn=(x,y))
                               ,u_par4(j_hatn=(x,y))
                               ,u_par5(j_hatn=add((x,y),j_2.vec()))
                              ))


i_2.addListeners(lambda x, y: ( u_par2(i_hatn=(x,y))
                               ,u_par4(i_hatn=(x,y))
                               ,u_par5(i_hatn=add((x,y),i_1.vec()))
                              ))

j_2.addListeners(lambda x, y: ( u_par2(j_hatn=(x,y))
                               ,u_par3(j_hatn=(x,y))
                               ,u_par5(j_hatn=add((x,y),j_1.vec()))
                              ))


ax_x = Axis(scale=sc_x)
ax_y = Axis(scale=sc_y, orientation='vertical')
fig1 = Figure(marks=i_1.marks+j_1.marks, axes=[ax_x, ax_y])
fig2 = Figure(marks=i_2.marks+j_2.marks, axes=[ax_x, ax_y])
fig3 = Figure(marks=[par1,par2,par3,par4,par5], axes=[ax_x, ax_y])
HBox([fig1,fig2,fig3])